<a href="https://colab.research.google.com/github/ShimilSBabu/Tensorflow-Model-Sub-Classing-API-Training/blob/main/facial_expression_detection_subclassing_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

# Constants

In [ ]:
dataset_link = "https://www.kaggle.com/datasets/ahmedmoorsy/facial-expression"
filname = '../content/facial-expression/fer2013/fer2013.csv'
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
names=['emotion','pixels','usage']

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import opendatasets as od

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.utils import load_img, img_to_array

from sklearn.model_selection import train_test_split

import keras
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Input, Layer
from keras.layers import Dense, Activation, Dropout, Flatten, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers import BatchNormalization
from keras import backend as K

# Download the Dataset

In [ ]:
od.download(dataset_link)

In [ ]:
# get the data
df=pd.read_csv(filname, names=names, na_filter=False)
im=df['pixels']
df.head(10)

In [ ]:
def getData(filname):
    # images are 48x48
    # N = 35887
    Y = []
    X = []
    first = True
    for line in open(filname):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])

    X, Y = np.array(X) / 255.0, np.array(Y)
    return X, Y


In [ ]:
X, Y = getData(filname)
num_class = len(set(Y))
print(num_class)

In [ ]:
# keras with tensorflow backend
N, D = X.shape
X = X.reshape(N, 48, 48, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
y_train = (np.arange(num_class) == y_train[:, None]).astype(np.float32)
y_test = (np.arange(num_class) == y_test[:, None]).astype(np.float32)

In [ ]:
class custom_cnn_block(Model):
  def __init__(self, channels, kernel_size = 3, batch_norm = 0, max_pool = 0):
    super(custom_cnn_block, self).__init__()
    # self.conv_1 = Conv2D(channels, (kernel_size, 1), padding = 'same')
    # self.conv_2 = Conv2D(channels, (1, kernel_size), padding = 'same')
    self.conv_1 = Conv2D(channels, kernel_size, padding = 'same')
    self.conv_2 = Conv2D(channels, kernel_size, padding = 'same')
    self.batch_norm_1 = BatchNormalization()
    self.maxpooling_1 = MaxPooling2D(pool_size=(2, 2))
    self.batch_norm = batch_norm
    self.max_pool = max_pool

  def call(self, input_tensor, training = False):
    x = self.conv_1(input_tensor)
    x = self.conv_2(x)

    if self.batch_norm:
      x = self.batch_norm_1(x, training = training)
    if self.max_pool:
      x = self.maxpooling_1(x)
    
    return x

In [ ]:
class custom_subclassing_model(tf.keras.Model):
  def __init__(self, num_classes = 7):
    super(custom_subclassing_model, self).__init__()
    self.block_1 = custom_cnn_block(64, kernel_size = 5)
    self.block_2 = custom_cnn_block(64, kernel_size = 5, batch_norm = 1)
    self.block_3 = custom_cnn_block(128, kernel_size = 3)
    self.block_4 = custom_cnn_block(128, kernel_size = 3, batch_norm = 1, max_pool = 1)
    self.block_5 = custom_cnn_block(256, kernel_size = 3)
    self.pool = GlobalAveragePooling2D()
    self.dense_1 = Dense(units = 128)
    self.classifier = Dense(num_classes)

  def call(self, input_tensor, training = False):
  
    x = self.block_1(input_tensor, training = training)
    x = self.block_2(x, training = training)
    x = tf.nn.relu(x)
    x = self.block_3(x, training = training)
    x = self.block_4(x, training = training)
    x = tf.nn.relu(x)
    x = self.block_5(x, training = training)
    x = tf.nn.relu(x)
    # x = Dropout(x, 0.3)
    x = self.pool(x)
    # x = Dropout(x, 0.3)
    x = self.dense_1(x)
    x = self.classifier(x)
    x = tf.nn.softmax(x)
    # x = Activation('softmax')(x)

    return x

In [ ]:
model = custom_subclassing_model()
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
# model.summary()


In [ ]:
path_model='model_filter.h5' # save model at this location after each epoch
K.clear_session() # destroys the current graph and builds a new one
# model=my_model() # create the model
K.set_value(model.optimizer.lr,1e-2) # set the learning rate

In [ ]:
# fit the model
model.fit(X_train,     
            y_train, 
            batch_size=64, 
            epochs=35, 
            )

In [ ]:
objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
y_pos = np.arange(len(objects))
print(y_pos)

In [ ]:
def emotion_analysis(emotions):
    objects = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, emotions, align='center', alpha=0.9)
    plt.tick_params(axis='x', which='both', pad=10,width=4,length=10)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')
    
plt.show()

In [ ]:
y_pred=model.predict(X_test)
#print(y_pred)
y_test.shape

**Real Time Expression Prediction**

In [ ]:
from skimage import io
image_path = input('Enter the image path : ')
# image_path = '/content/download.jpg'
img = load_img(image_path,color_mode = "grayscale", grayscale=True, target_size=(48, 48))
show_img= load_img(image_path, grayscale=False, target_size=(200, 200))
x = img_to_array(img)
x = np.expand_dims(x, axis = 0)

x /= 255

custom = model.predict(x)
#print(custom[0])
emotion_analysis(custom[0])

x = np.array(x, 'float32')
x = x.reshape([48, 48]);

plt.gray()
plt.imshow(show_img)
plt.show()

m=0.000000000000000000001
a=custom[0]
for i in range(0,len(a)):
    if a[i]>m:
        m=a[i]
        ind=i
        
print('Expression Prediction:',objects[ind])
        